This algorithm aims to compute the evolutionary strategy of delayed dispersal of our model. To do so, we evaluate the inclusive fitness change when we increase slightly the level of delayed dispersal. If this change is positive, we increment level of delayed dispesal by a small amount. We do this process for the level of delayed dispersal in low-quality group h_x and high quality groups h_y. When compute the inclusive change of both delayed dispersal strategies until they are in a tolerance of 0, or are on the boundary of the [0,1]^2 square. We assume we found the evolutionary equilibrium (h_x,h_y) when they are in a tolerance of 0.

# Getting started: libraries, parameters and functions

In [6]:
#Libraries
using LinearAlgebra
using Plots
using LaTeXStrings
using Profile
using Random
using CSV
using DataFrames

In [7]:
#Parameters

#survival
s_u = 0.5
s_x = 0.5
s_y = 0.5
s_ax = 0.5
s_ay = 0.5

#fecundity
b_x = 3
b_y = 3

phi = 0.1
M = 1

#Male and female reproduction:
theta_x(x,y) = (1-phi)*x/(x+y)
theta_y(x,y) = (1-phi)*y/(x+y)
psi_x(x,y) = 1+phi+theta_x(x,y)
psi_y(x,y) = 1+phi+theta_y(x,y)

#competition
p_u = 0.5 #floater ability
p_a = 0.25 #helper ability
a = 2 #strength of the density
#Actual probability to find a breeding spot during a time step
p_u_t(x,y) = p_u/(1+a*(x+y))
p_a_t(x,y) = p_a/(1+a*(x+y))

#Cooperation levels
h_x = 0
h_y = 0
h_x_bar = 0
h_y_bar = 0

#Parameter dictionary
parameters = Dict("b_x"=>b_x,"b_y"=>b_y,"s_u"=>s_u,"s_ax"=>s_ax,"s_ay"=>s_ay,"s_x"=>s_x,"s_y"=>s_y,"phi"=>phi,"M"=>M,"a"=>a,"p_u"=>p_u,"p_a"=>p_a,"h_x" =>h_x, "h_y" =>h_y,"h_x_bar" =>h_x_bar, "h_y_bar" =>h_y_bar)


Dict{String, Real} with 16 entries:
  "s_u"     => 0.5
  "h_x_bar" => 0
  "h_x"     => 0
  "p_u"     => 0.5
  "s_y"     => 0.5
  "a"       => 2
  "h_y_bar" => 0
  "M"       => 1
  "phi"     => 0.1
  "s_ay"    => 0.5
  "b_x"     => 3
  "h_y"     => 0
  "b_y"     => 3
  "s_x"     => 0.5
  "p_a"     => 0.25
  "s_ax"    => 0.5

Usefull functions: Wild type population matrix A, mutant population type matrix M, computations of the eigenvectors mu (right) and nu (left) and relatedness coefficient.

In [8]:
#Population matrix
function B(x,y,parameters) #x and y are the number of breeders in low quality (x) and high quality (y) groups
    #First we retrieve the parameters from the dictionnary parameters
    b_x = parameters["b_x"]
    b_y = parameters["b_y"]
    s_u = parameters["s_u"]
    s_ax = parameters["s_ax"]
    s_ay = parameters["s_ay"]
    s_x = parameters["s_x"]
    s_y = parameters["s_y"]
    p_u = parameters["p_u"]
    p_a = parameters["p_a"]
    a = parameters["a"]
    M = parameters["M"]
    phi = parameters["phi"]
    h_x = parameters["h_x"]
    h_y = parameters["h_y"]
    h_x_bar = parameters["h_x_bar"]
    h_y_bar = parameters["h_y_bar"]
    
    #We use the parameters to compute the functions
    p_u_t(x,y) = p_u/(1+a*(x+y))
    p_a_t(x,y) = p_a/(1+a*(x+y))
    theta_x(x,y) = (1-phi)*x/(x+y)
    theta_y(x,y) = (1-phi)*y/(x+y)
    psi_x(x,y) = theta_x(x,y)+1+phi
    psi_y(x,y) = theta_y(x,y)+1+phi
    T_x(h_x) = 1-exp(-h_x*b_x)
    T_y(h_y) = exp(-h_y*b_y)
    
    #Finally, we return the matrix A(x,y):
    return [s_u*(1-p_u_t(x,y)) 0 0 (1-h_x)*s_u*(1-p_u_t(x,y))+h_x*(T_x(h_x_bar)*s_ay+(1-T_x(h_x_bar))*s_ax)*(1-p_a_t(x,y)) (1-h_y)*s_u*(1-p_u_t(x,y))+h_y*(T_y(h_y_bar)*s_ax+(1-T_y(h_y_bar))*s_ay)*(1-p_a_t(x,y));
    s_u*p_u_t(x,y) (1-T_x(h_x_bar))*s_x T_y(h_y_bar)*s_x (1-h_x)*s_u*p_u_t(x,y)+h_x*(T_x(h_x_bar)*s_ay+(1-T_x(h_x_bar))*s_ax)*p_a_t(x,y) (1-h_y)*s_u*p_u_t(x,y)+h_y*(T_y(h_y_bar)*s_ax+(1-T_y(h_y_bar))*s_ay)*p_a_t(x,y);
    0 T_x(h_x_bar)*s_y (1-T_y(h_y_bar))*s_y 0 0
    s_u*p_u_t(x,y)*b_x (1-T_x(h_x_bar))*s_x*b_x T_y(h_y_bar)*s_x*b_x ((1-h_x)*s_u*p_u_t(x,y)+h_x*(T_x(h_x_bar)*s_ay+(1-T_x(h_x_bar))*s_ax)*p_a_t(x,y))*b_x ((1-h_y)*s_u*p_u_t(x,y)+h_y*(T_y(h_y_bar)*s_ax+(1-T_y(h_y_bar))*s_ay)*p_a_t(x,y))*b_x;
    0 T_x(h_x_bar)*s_y*b_y (1-T_y(h_y_bar))*s_y*b_y 0 0
    ]
end


function Mutant(x,y,parameters)
    #As for B, we retrieve the parameters from the dictionnary and compute the functions
    b_x = parameters["b_x"]
    b_y = parameters["b_y"]
    s_u = parameters["s_u"]
    s_ax = parameters["s_ax"]
    s_ay = parameters["s_ay"]
    s_x = parameters["s_x"]
    s_y = parameters["s_y"]
    p_u = parameters["p_u"]
    p_a = parameters["p_a"]
    a = parameters["a"]
    M = parameters["M"]
    phi = parameters["phi"]
    h_x = parameters["h_x"]
    h_y = parameters["h_y"]
    h_x_bar = parameters["h_x_bar"]
    h_y_bar = parameters["h_y_bar"]
    p_u_t(x,y) = p_u/(1+a*(x+y))
    p_a_t(x,y) = p_a/(1+a*(x+y))
    theta_x(x,y) = (1-phi)*x/(x+y)
    theta_y(x,y) = (1-phi)*y/(x+y)
    psi_x(x,y) = theta_x(x,y)+1+phi
    psi_y(x,y) = theta_y(x,y)+1+phi
    T_x(h_x) = 1-exp(-h_x*b_x)
    T_y(h_y) = exp(-h_y*b_y)
    
    #We return the value of the mutant matrix M(x,y)
    return [s_u*(1-p_u_t(x,y)) 0 0 (1-h_x)*s_u*(1-p_u_t(x,y))+h_x*(T_x(h_x_bar)*s_ay+(1-T_x(h_x_bar))*s_ax)*(1-p_a_t(x,y)) (1-h_y)*s_u*(1-p_u_t(x,y))+h_y*(T_y(h_y_bar)*s_ax+(1-T_y(h_y_bar))*s_ay)*(1-p_a_t(x,y));
    s_u*p_u_t(x,y) (1-T_x(h_x_bar))*s_x T_y(h_y_bar)*s_x (1-h_x)*s_u*p_u_t(x,y)+h_x*(T_x(h_x_bar)*s_ay+(1-T_x(h_x_bar))*s_ax)*p_a_t(x,y) (1-h_y)*s_u*p_u_t(x,y)+h_y*(T_y(h_y_bar)*s_ax+(1-T_y(h_y_bar))*s_ay)*p_a_t(x,y);
    0 T_x(h_x_bar)*s_y (1-T_y(h_y_bar))*s_y 0 0;
    s_u*p_u_t(x,y)*b_x/2*psi_x(x,y) (1-T_x(h_x_bar))*s_x*b_x/2*psi_x(x,y)+T_x(h_x_bar)*s_y*b_x/2*theta_x(x,y) T_y(h_y_bar)*s_x*b_x/2*psi_x(x,y)+(1-T_y(h_y_bar))*s_y*b_x/2*theta_x(x,y) ((1-h_x)*s_u*p_u_t(x,y)+h_x*(T_x(h_x_bar)*s_ay+(1-T_x(h_x_bar))*s_ax)*p_a_t(x,y))*b_x/2*psi_x(x,y) ((1-h_y)*s_u*p_u_t(x,y)+h_y*(T_y(h_y_bar)*s_ax+(1-T_y(h_y_bar))*s_ay)*p_a_t(x,y))*b_x/2*psi_x(x,y);
    s_u*p_u_t(x,y)*b_y/2*theta_y(x,y) T_x(h_x_bar)*s_y*b_y/2*psi_y(x,y)+(1-T_x(h_x_bar))*s_x*b_y/2*theta_y(x,y) (1-T_y(h_y_bar))*s_y*b_y/2*psi_y(x,y)+T_y(h_y_bar)*s_x*b_y/2*theta_y(x,y) ((1-h_x)*s_u*p_u_t(x,y)+h_x*(T_x(h_x_bar)*s_ay+(1-T_x(h_x_bar))*s_ax)*p_a_t(x,y))*b_y/2*theta_y(x,y) ((1-h_y)*s_u*p_u_t(x,y)+h_y*(T_y(h_y_bar)*s_ax+(1-T_y(h_y_bar))*s_ay)*p_a_t(x,y))*b_y/2*theta_y(x,y)
    ]
end



function right_eigenvector(parameters)
    #We suppose a random value for the equilibrium of the mutant matrix
    mu = rand!(zeros(5,1))
    #And compute the equilibrium value until we find a value that satisfies the tolerance (1e^-8)
    while sum(abs.(mu-B(mu[2],mu[3],parameters)*mu))>1e-8
        mu = B(mu[2],mu[3],parameters)*mu
    end
    return mu
end

function left_eigenvector(parameters,mu)
    #We suppose a random value for the left eigenvalue of the mutant matrix
    nu = rand!(zeros(1,5))
    #And compute the left eigenvector value until we find a value that satisfies the tolerance (1e^-8)
    while sum(abs.(nu-nu*Mutant(mu[2],mu[3],parameters)))>1e-8
        nu = nu*Mutant(mu[2],mu[3],parameters)
        nu /= sum(nu)
    end
    return nu
end

function R_parent(parameters)
    return 1/2*(1+parameters["phi"]) 
end

function R_siblings_x(parameters)
    b_x = parameters["b_x"] 
    M = parameters["M"]
    phi = parameters["phi"]
    return  (1-1/b_x)*(phi^2+phi*(1-phi)+(1-phi)^2*(1/4+1/(4*M)))+1/b_x
end

function R_siblings_y(parameters)
    M = parameters["M"]
    phi = parameters["phi"]
    b_y = parameters["b_y"]   
    return  (1-1/b_y)*(phi^2+phi*(1-phi)+(1-phi)^2*(1/4+1/(4*M)))+1/b_y
end;

Now that we have our useful function, we can compute the evolution of the delayed dispersal rates h_x, h_y

In [7]:
function stay_help_evol_sub(parameters)
    #initialization of the inclusive fitness change so the algorithm does not stop at the first iteration
    dW_x = 1
    dW_y = 1
    #delta h_x, delta h_y, the increment of h_x and h_y.
    dhx = 1e-4 
    dhy = 1e-4
    #Tolerance of zero for the inclusive fitness change
    tol = 1e-8
    parameters["h_x"] = 0
    parameters["h_x_bar"] = 0
    parameters["h_y"] = 0
    parameters["h_y_bar"] = 0
    i = 0        
    reason = "too many iterations"
    #storing values for delayed dispersal level and inclusive fitness changes: this allows us to control the changes in order to stop the algorithm when appropriate
    hx_mem = 10
    hy_mem = 10
    dwx_mem = 10
    dwy_mem = 10
    
    #Evolution
    #We loop the computations for a sufficient amount of time until the algorithm stops for the different reasons presented in the article, see below.
    while i<1000000
        reason = "too many iterations"
        
        #Computation of the eigenvectors mu and nu
        mu = right_eigenvector(parameters)
        nu = left_eigenvector(parameters,mu)
        
        #h_x evolution
        #We assume h_x is slightly higher and evaluate the change in the fitness of the individual, its breeder, the other helpers on the nest using a finite difference scheme
        parameters["h_x"] += dhx 
        W_indiv_x = nu*Mutant(mu[2],mu[3],parameters)*[0,0,0,mu[4],0]
    
        parameters["h_x"] -= 2*dhx
        W_indiv_x -= nu*Mutant(mu[2],mu[3],parameters)*[0,0,0,mu[4],0]
        parameters["h_x"] += dhx

        parameters["h_x_bar"] += dhx
        W_parent_x = nu*Mutant(mu[2],mu[3],parameters)*[0,mu[2],0,0,0]
        
       
        W_sibs_x = nu*Mutant(mu[2],mu[3],parameters)*[0,0,0,mu[4],0]

        parameters["h_x_bar"] -= 2*dhx
        W_parent_x -= nu*Mutant(mu[2],mu[3],parameters)*[0,mu[2],0,0,0]
        

        
        W_sibs_x -= nu*Mutant(mu[2],mu[3],parameters)*[0,0,0,mu[4],0]
        parameters["h_x_bar"] += dhx
        
            
        #h_y, same process as h_x
        parameters["h_y"] += dhy
        W_indiv_y = nu*Mutant(mu[2],mu[3],parameters)*[0,0,0,0,mu[5]]
        parameters["h_y"] -= 2*dhy
        W_indiv_y -= nu*Mutant(mu[2],mu[3],parameters)*[0,0,0,0,mu[5]]
        parameters["h_y"] += dhy

        parameters["h_y_bar"] += dhy
        W_parent_y = nu*Mutant(mu[2],mu[3],parameters)*[0,0,mu[3],0,0]
        W_sibs_y = nu*Mutant(mu[2],mu[3],parameters)*[0,0,0,0,mu[5]]

        parameters["h_y_bar"] -= 2*dhy
        W_parent_y -= nu*Mutant(mu[2],mu[3],parameters)*[0,0,mu[3],0,0]
        W_sibs_y -= nu*Mutant(mu[2],mu[3],parameters)*[0,0,0,0,mu[5]]
        parameters["h_y_bar"] += dhy
        
     
        
        #If the subordinate controls stay-and-help
        dW_x = R_parent(parameters)*W_parent_x[1]+W_indiv_x[1]+R_siblings_x(parameters)*W_sibs_x[1]
        dW_y = R_parent(parameters)*W_parent_y[1]+W_indiv_y[1]+R_siblings_y(parameters)*W_sibs_y[1]
        
        #We store the value of hx, hy and the inclusive fitness changes
        hx_mem = parameters["h_x"]
        hy_mem = parameters["h_y"]
        dwx_mem = dW_x
        dwy_mem = dW_y
        #We update hx and hy
        parameters["h_x"] += dW_x*50
        parameters["h_x_bar"] += dW_x*50
        parameters["h_y"] += dW_y*50
        parameters["h_y_bar"] += dW_y*50

        i+=1
        #We keep both values between 0 and 1 since they are probabilities
        parameters["h_x"] = clamp(parameters["h_x"],0,1)
        parameters["h_x_bar"] = clamp(parameters["h_x_bar"],0,1)
        parameters["h_y"] = clamp(parameters["h_y"],0,1)
        parameters["h_y_bar"] = clamp(parameters["h_y_bar"],0,1)
        
        #the algorithm stops if:
        #1. The inclusive fitness change is within a tolerance of 0
        if abs(dW_x)<tol && abs(dW_y)<tol
            #If we run for too long, we break the loop
            reason = "css found"
            break
        end
        #2. Both hx, hy values are repeated (this can happen at the border of the [0,1]x[0,1] square)
        
        if parameters["h_x"] == hx_mem && parameters["h_y"] == hy_mem
             reason = "stagnation"
            break
        end
       
        #3. One value is repeated while the other remains an internal equilibrium: the change for both values is thus
        #null/not significant
        if parameters["h_x"] == hx_mem && abs(dW_y)<tol && abs(dwy_mem)<tol
            reason = "limit"
            break
        end
        
        if parameters["h_y"] == hy_mem && abs(dW_x)<tol && abs(dwx_mem)<tol
            reason = "limit"
            break
        end    
        
    end
    
    #Now that hx and hy are computed, we evaluate the reproductive value of a surviving helper, cost of help at evolutionary equilibrium and probability of establishment:
    #First the eigenvectors:
    mu = right_eigenvector(parameters)
    nu = left_eigenvector(parameters,mu)
    
    #Reproductive value of a surviving helper
    nuH = (1-p_a_t(mu[2],mu[3]))*nu[1]+p_a_t(mu[2],mu[3])*(nu[2]+(1+phi)*b_x/2*nu[4]+(1-phi)*mu[2]/(mu[2]+mu[3])*b_x/2*nu[4]+(1-phi)*mu[3]/(mu[2]+mu[3])*b_y/2*nu[5])
    
    #Cost of help
    parameters["h_x"] += dhx
    W_indiv_x = nu*Mutant(mu[2],mu[3],parameters)*[0,0,0,mu[4],0]
    parameters["h_x"] -= 2*dhx
    W_indiv_x -= nu*Mutant(mu[2],mu[3],parameters)*[0,0,0,mu[4],0]
    parameters["h_x"] += dhx
    cost = W_indiv_x[1]
    
    #Probability of establishment
    P2 = parameters["s_u"]*p_u_t(mu[2],mu[3])/(1-parameters["s_u"]*(1-p_u_t(mu[2],mu[3])))
    
    return parameters, P2, nuH, cost, reason
end



stay_help_evol_sub (generic function with 1 method)

In [9]:
function evol_dd(parameters,ctrl)
    #initialization
    dW_x = 1
    dW_y = 1
    signx = 1
    signy = 1
    #delta h_x, delta h_y:
    dhx = 1e-4
    dhy = 1e-4
    tol = 1e-8
    parameters["h_x"] = 0
    parameters["h_x_bar"] = 0
    parameters["h_y"] = 0
    parameters["h_y_bar"] = 0
    i = 0        
    reason = "too many iterations"

    hx_mem = 10
    hy_mem = 10
    dwx_mem = 10
    dwy_mem = 10
    #Evolution
    #We run the simulation for a sufficient amount of time until the simulation stop
    while i<1000000
        reason = "too many iterations"

        #Eigenvectors mu and nu
        mu = right_eigenvector(parameters)
        nu = left_eigenvector(parameters,mu)
        #h_x evolution
        parameters["h_x"] += dhx #We assume h_x is slightly higher and evaluate the change in the fitness of the individual, its breeder, the other late dispersers
        W_indiv_x = nu*Mutant(mu[2],mu[3],parameters)*[0,0,0,mu[4],0]
        parameters["h_x"] -= 2*dhx
        W_indiv_x -= nu*Mutant(mu[2],mu[3],parameters)*[0,0,0,mu[4],0]
        parameters["h_x"] += dhx

        parameters["h_x_bar"] += dhx
        W_parent_x = nu*Mutant(mu[2],mu[3],parameters)*[0,mu[2],0,0,0]
        W_sibs_x = nu*Mutant(mu[2],mu[3],parameters)*[0,0,0,mu[4],0]

        parameters["h_x_bar"] -= 2*dhx
        W_parent_x -= nu*Mutant(mu[2],mu[3],parameters)*[0,mu[2],0,0,0]
        W_sibs_x -= nu*Mutant(mu[2],mu[3],parameters)*[0,0,0,mu[4],0]
        parameters["h_x_bar"] += dhx
        
        #h_y, same process as h_x
        parameters["h_y"] += dhy
        W_indiv_y = nu*Mutant(mu[2],mu[3],parameters)*[0,0,0,0,mu[5]]
        parameters["h_y"] -= 2*dhy
        W_indiv_y -= nu*Mutant(mu[2],mu[3],parameters)*[0,0,0,0,mu[5]]
        parameters["h_y"] += dhy

        parameters["h_y_bar"] += dhy
        W_parent_y = nu*Mutant(mu[2],mu[3],parameters)*[0,0,mu[3],0,0]
        W_sibs_y = nu*Mutant(mu[2],mu[3],parameters)*[0,0,0,0,mu[5]]

        parameters["h_y_bar"] -= 2*dhy
        W_parent_y -= nu*Mutant(mu[2],mu[3],parameters)*[0,0,mu[3],0,0]
        W_sibs_y -= nu*Mutant(mu[2],mu[3],parameters)*[0,0,0,0,mu[5]]
        parameters["h_y_bar"] += dhy
        
        
        if ctrl=="sub"
            dW_x = R_parent(parameters)*W_parent_x[1]+W_indiv_x[1]+R_siblings_x(parameters)*W_sibs_x[1]
            dW_y = R_parent(parameters)*W_parent_y[1]+W_indiv_y[1]+R_siblings_y(parameters)*W_sibs_y[1]
        else
            dW_x = W_parent_x[1]+(W_sibs_x[1]+W_indiv_x[1])*R_parent(parameters)
            dW_y = W_parent_y[1]+(W_sibs_y[1]+W_indiv_y[1])*R_parent(parameters)
        end
        hx_mem = parameters["h_x"]
        hy_mem = parameters["h_y"]
        dwx_mem = dW_x
        dwy_mem = dW_y
        parameters["h_x"] += dW_x*50
        parameters["h_x_bar"] += dW_x*50
        parameters["h_y"] += dW_y*50
        parameters["h_y_bar"] += dW_y*50

        i+=1
        #We keep both values between 0 and 1 since they are probabilities
        parameters["h_x"] = clamp(parameters["h_x"],0,1)
        parameters["h_x_bar"] = clamp(parameters["h_x_bar"],0,1)
        parameters["h_y"] = clamp(parameters["h_y"],0,1)
        parameters["h_y_bar"] = clamp(parameters["h_y_bar"],0,1)
        
        #the algorithm stops if:
        #1. The inclusive fitness change is within a tolerance of 0
        if abs(dW_x)<tol && abs(dW_y)<tol
            #If we run for too long, we break the loop
            reason = "css found"
            break
        end
        #2. Both values are repeated (this can happen at the border of the [0,1]x[0,1] square)
        
        if parameters["h_x"] == hx_mem && parameters["h_y"] == hy_mem
             reason = "stagnation"
            break
        end
       
        #3. One value is repeated while the other remains an internal equilibrium: the change for both values is thus
        #null/not significant
        if parameters["h_x"] == hx_mem && abs(dW_y)<tol && abs(dwy_mem)<tol
            reason = "limit"
            break
        end
        
        if parameters["h_y"] == hy_mem && abs(dW_x)<tol && abs(dwx_mem)<tol
            reason = "limit"
            break
        end    
        
    end
    #Evaluation of group augmentation and cost of delayed dispersal at evolutionary equilibrium:
    eq = right_eigenvector(parameters)
    mu = eq
    nu = left_eigenvector(parameters,mu)
    #Group augmentation effect
    GA = (1-p_a_t(mu[2],mu[3]))*nu[1]+p_a_t(mu[2],mu[3])*(nu[2]+(1+phi)*b_x/2*nu[4]+(1-phi)*mu[2]/(mu[2]+mu[3])*b_x/2*nu[4]+(1-phi)*mu[3]/(mu[2]+mu[3])*b_y/2*nu[5])
    #Probability of establishment
    P2 = parameters["s_u"]*p_u_t(mu[2],mu[3])/(1-parameters["s_u"]*(1-p_u_t(mu[2],mu[3])))
    #Cost of delaying dispersal
    parameters["h_x"] += dhx
    W_indiv_x = nu*Mutant(mu[2],mu[3],parameters)*[0,0,0,mu[4],0]
    parameters["h_x"] -= 2*dhx
    W_indiv_x -= nu*Mutant(mu[2],mu[3],parameters)*[0,0,0,mu[4],0]
    parameters["h_x"] += dhx
    cost = W_indiv_x[1]
    return parameters, P2, GA, cost, reason
end

evol_dd (generic function with 1 method)

# Simulations

## Figure 2: impact of selfing rate phi on delayed dispersal

In [7]:
#We first set the parameters
parameters["s_ax"] = 0.5
parameters["s_ay"] = 0.6
parameters["s_u"] = 0.5
parameters["s_x"] = 0.5
parameters["s_y"] = 0.5
parameters["b_x"] = 3
parameters["b_y"] = 3
parameters["phi"] = 0.1
N = 30

#We initialize our data
phi_csv = zeros(N^2)
db_csv = zeros(N^2)
hx_csv = zeros(N^2)
hy_csv = zeros(N^2)
P_csv = zeros(N^2)
GA_csv = zeros(N^2)
cost_csv = zeros(N^2)
parameters_csv = Array{Dict}(undef, N^2)
reason_csv = Array{String}(undef, N^2)
j = 1

#We loop over the parameters we are interested in, here fecundity benefits and phi
for db in range(0,2,N)
    parameters["b_y"] = parameters["b_x"]+db
    for phi in range(0,1,N)
        parameters["phi"] = phi
        parameters, P2, GA, cost, reason = evol_dd(parameters,"sub")
        hx_csv[j] = parameters["h_x"]
        hy_csv[j] = parameters["h_y"]
        phi_csv[j] = phi
        db_csv[j] = db
        GA_csv[j] = GA
        cost_csv[j] = cost
        reason_csv[j] = reason
        parameters_csv[j] = parameters
        j += 1
    end
end
#Finally, we write the result in a csv file
csv_data = DataFrame(db = db_csv, phi = phi_csv, h_x = hx_csv, h_y = hy_csv, cost=cost_csv, GA = GA_csv, parameters = parameters_csv, reason = reason_csv )

CSV.write("result_phi_b_sub.csv",csv_data)

"result_phi_b_GA_sub_3.csv"

## Figure 3: relationship between delayed dispersal and probability of establishment

Panel A: fecundity

In [13]:
parameters["s_x"] = 0.5
parameters["s_y"] = 0.5
parameters["s_ax"] = 0.5
parameters["s_ay"] = 0.5
parameters["s_u"] = 0.5
parameters["phi"] = 0.1
N = 20
b_csv = zeros(N^2)
db_csv = zeros(N^2)
hx_csv = zeros(N^2)
hy_csv = zeros(N^2)
P2_csv = zeros(N^2)
parameters_csv = Array{Dict}(undef, N^2)
reason_csv = Array{String}(undef, N^2)
j = 1
for b in range(3,7,N)
    parameters["b_x"] = b
    for db in range(0,5,N)
        parameters["b_y"] = parameters["b_x"]+db
        parameters, P2, GA, cost, reason = evol_dd(parameters,"sub")
        hx_csv[j] = parameters["h_x"]
        hy_csv[j] = parameters["h_y"]
        b_csv[j] = b
        db_csv[j] = db
        P2_csv[j] = P2
        reason_csv[j] = reason
        parameters_csv[j] = parameters
        j += 1
    end
end
csv_data = DataFrame(db = db_csv, b = b_csv, h_x = hx_csv, h_y = hy_csv, P2 = P2_csv, parameters = parameters_csv, reason = reason_csv )
CSV.write("result_b_sub.csv",csv_data)

"result_b_sub.csv"

Panel B: survival of the breeders

In [ ]:
parameters["b_x"] = 3
parameters["b_y"] = 3
parameters["s_ax"] = 0.5
parameters["s_ay"] = 0.5
parameters["s_u"] = 0.5
parameters["phi"] = 0.1
N = 20
s_csv = zeros(N^2)
ds_csv = zeros(N^2)
hx_csv = zeros(N^2)
hy_csv = zeros(N^2)
P2_csv = zeros(N^2)
parameters_csv = Array{Dict}(undef, N^2)
reason_csv = Array{String}(undef, N^2)
j = 1
for s in range(0.3,0.7,N)
    parameters["s_x"] = s
    for ds in range(0,0.2,N)
        parameters["s_y"] = parameters["s_x"]+ds
        parameters, P2, GA, cost, reason = evol_dd(parameters,"sub")
        hx_csv[j] = parameters["h_x"]
        hy_csv[j] = parameters["h_y"]
        s_csv[j] = s
        ds_csv[j] = ds    
        P2_csv[j] = P2
        reason_csv[j] = reason
        parameters_csv[j] = parameters
        j += 1
    end
end
csv_data = DataFrame(ds = ds_csv, s = s_csv, h_x = hx_csv, h_y = hy_csv, P2 = P2_csv, parameters = parameters_csv, reason = reason_csv )
CSV.write("result_s_sub.csv",csv_data)

Panel C: survival of the subordinates

In [15]:
parameters["b_x"] = 3
parameters["b_y"] = 5
parameters["s_x"] = 0.5
parameters["s_y"] = 0.5
parameters["s_u"] = 0.5
parameters["phi"] = 0.1
N = 20
s_csv = zeros(N^2)
ds_csv = zeros(N^2)
hx_csv = zeros(N^2)
hy_csv = zeros(N^2)
P2_csv = zeros(N^2)
parameters_csv = Array{Dict}(undef, N^2)
reason_csv = Array{String}(undef, N^2)
j = 1
for s in range(0.5,0.7,N)
    parameters["s_u"] = s
    parameters["s_ax"] = s
    for ds in range(0,0.15,N)
        parameters["s_ay"] = parameters["s_ax"]+ds
        parameters, P2, GA, cost, reason = stay_help_evol_sub(parameters)
        hx_csv[j] = parameters["h_x"]
        hy_csv[j] = parameters["h_y"]
        s_csv[j] = s
        ds_csv[j] = ds    
        P2_csv[j] = P2
        reason_csv[j] = reason
        parameters_csv[j] = parameters
        j += 1
    end
end
csv_data = DataFrame(ds = ds_csv, s = s_csv, h_x = hx_csv, h_y = hy_csv, P2 = P2_csv, parameters = parameters_csv, reason = reason_csv )
CSV.write("result_say_sub.csv",csv_data)

"result_say_sub.csv"

## Figure 4: breeder-subordinate conflict

Fecundity without GA

In [10]:
parameters["s_x"] = 0.5
parameters["s_y"] = 0.5
parameters["s_ax"] = 0.5
parameters["s_ay"] = 0.5
parameters["s_u"] = 0.5
parameters["phi"] = 0.1
N = 50
b_csv = zeros(N*3)
db_csv = zeros(N*3)
hx_csv = zeros(N*3)
hy_csv = zeros(N*3)
P2_csv = zeros(N*3)
parameters_csv = Array{Dict}(undef, N*3)
reason_csv = Array{String}(undef, N*3)
j = 1
for b in [4,5,6]
    parameters["b_x"] = b
    for db in range(0,5,N)
        parameters["b_y"] = parameters["b_x"]+db
        parameters, P2, GA, cost, reason = evol_dd(parameters,"sub")
        hx_csv[j] = parameters["h_x"]
        hy_csv[j] = parameters["h_y"]
        b_csv[j] = b
        db_csv[j] = db
        P2_csv[j] = P2
        reason_csv[j] = reason
        parameters_csv[j] = parameters
        j += 1
    end
end
csv_data = DataFrame(db = db_csv, b = b_csv, h_x = hx_csv, h_y = hy_csv, P2 = P2_csv, parameters = parameters_csv, reason = reason_csv )
CSV.write("result_b_sub_3.csv",csv_data)

"result_b_bre_4.csv"

In [ ]:
parameters["s_x"] = 0.5
parameters["s_y"] = 0.5
parameters["s_ax"] = 0.5
parameters["s_ay"] = 0.5
parameters["s_u"] = 0.5
parameters["phi"] = 0.1
N = 50
b_csv = zeros(N*3)
db_csv = zeros(N*3)
hx_csv = zeros(N*3)
hy_csv = zeros(N*3)
P2_csv = zeros(N*3)
parameters_csv = Array{Dict}(undef, N*3)
reason_csv = Array{String}(undef, N*3)
j = 1
for b in [4,5,6]
    parameters["b_x"] = b
    for db in range(0,5,N)
        parameters["b_y"] = parameters["b_x"]+db
        parameters, P2, GA, cost, reason = evol_dd(parameters,"bre")
        hx_csv[j] = parameters["h_x"]
        hy_csv[j] = parameters["h_y"]
        b_csv[j] = b
        db_csv[j] = db
        P2_csv[j] = P2
        reason_csv[j] = reason
        parameters_csv[j] = parameters
        j += 1
    end
end
csv_data = DataFrame(db = db_csv, b = b_csv, h_x = hx_csv, h_y = hy_csv, P2 = P2_csv, parameters = parameters_csv, reason = reason_csv )
CSV.write("result_b_bre_3.csv",csv_data)

Fecundity with GA

In [ ]:
parameters["s_x"] = 0.5
parameters["s_y"] = 0.5
parameters["s_ax"] = 0.5
parameters["s_ay"] = 0.6
parameters["s_u"] = 0.5
parameters["phi"] = 0.1
N = 50
b_csv = zeros(N*3)
db_csv = zeros(N*3)
hx_csv = zeros(N*3)
hy_csv = zeros(N*3)
P2_csv = zeros(N*3)
parameters_csv = Array{Dict}(undef, N*3)
reason_csv = Array{String}(undef, N*3)
j = 1
for b in [4,5,6]
    parameters["b_x"] = b
    for db in range(0,5,N)
        parameters["b_y"] = parameters["b_x"]+db
        parameters, P2, GA, cost, reason = evol_dd(parameters,"sub")
        hx_csv[j] = parameters["h_x"]
        hy_csv[j] = parameters["h_y"]
        b_csv[j] = b
        db_csv[j] = db
        P2_csv[j] = P2
        reason_csv[j] = reason
        parameters_csv[j] = parameters
        j += 1
    end
end
csv_data = DataFrame(db = db_csv, b = b_csv, h_x = hx_csv, h_y = hy_csv, P2 = P2_csv, parameters = parameters_csv, reason = reason_csv )
CSV.write("result_b_GA_sub_3.csv",csv_data)

In [ ]:
parameters["s_x"] = 0.5
parameters["s_y"] = 0.5
parameters["s_ax"] = 0.5
parameters["s_ay"] = 0.6
parameters["s_u"] = 0.5
parameters["phi"] = 0.1
N = 50
b_csv = zeros(N*3)
db_csv = zeros(N*3)
hx_csv = zeros(N*3)
hy_csv = zeros(N*3)
P2_csv = zeros(N*3)
parameters_csv = Array{Dict}(undef, N*3)
reason_csv = Array{String}(undef, N*3)
j = 1
for b in [4,5,6]
    parameters["b_x"] = b
    for db in range(0,5,N)
        parameters["b_y"] = parameters["b_x"]+db
        parameters, P2, GA, cost, reason = evol_dd(parameters,"bre")
        hx_csv[j] = parameters["h_x"]
        hy_csv[j] = parameters["h_y"]
        b_csv[j] = b
        db_csv[j] = db
        P2_csv[j] = P2
        reason_csv[j] = reason
        parameters_csv[j] = parameters
        j += 1
    end
end
csv_data = DataFrame(db = db_csv, b = b_csv, h_x = hx_csv, h_y = hy_csv, P2 = P2_csv, parameters = parameters_csv, reason = reason_csv )
CSV.write("result_b_GA_bre_3.csv",csv_data)

Survival without GA

In [6]:
parameters["b_x"] = 3
parameters["b_y"] = 3
parameters["s_u"] = 0.5
parameters["s_ax"] = 0.5
parameters["s_ay"] = 0.5
parameters["phi"] = 0.1
N = 50
s_csv = zeros(N*3)
ds_csv = zeros(N*3)
hx_csv = zeros(N*3)
hy_csv = zeros(N*3)
P2_csv = zeros(N*3)
parameters_csv = Array{Dict}(undef, N*3)
reason_csv = Array{String}(undef, N*3)
j = 1
for s in [0.3,0.5,0.7]
    parameters["s_x"] = s
    for ds in range(0,0.2,N)
        parameters["s_y"] = parameters["s_x"]+ds
        parameters, P2, GA, cost, reason = evol_dd(parameters,"sub")
        hx_csv[j] = parameters["h_x"]
        hy_csv[j] = parameters["h_y"]
        s_csv[j] = s
        ds_csv[j] = ds    
        P2_csv[j] = P2
        reason_csv[j] = reason
        parameters_csv[j] = parameters
        j += 1
    end
end
csv_data = DataFrame(ds = ds_csv, s = s_csv, h_x = hx_csv, h_y = hy_csv, P2 = P2_csv, parameters = parameters_csv, reason = reason_csv )
CSV.write("result_s_sub_3.csv",csv_data)

"result_s_bre3.csv"

In [ ]:
parameters["b_x"] = 3
parameters["b_y"] = 3
parameters["s_u"] = 0.5
parameters["s_ax"] = 0.5
parameters["s_ay"] = 0.5
parameters["phi"] = 0.1
N = 50
s_csv = zeros(N*3)
ds_csv = zeros(N*3)
hx_csv = zeros(N*3)
hy_csv = zeros(N*3)
P2_csv = zeros(N*3)
parameters_csv = Array{Dict}(undef, N*3)
reason_csv = Array{String}(undef, N*3)
j = 1
for s in [0.3,0.5,0.7]
    parameters["s_x"] = s
    for ds in range(0,0.2,N)
        parameters["s_y"] = parameters["s_x"]+ds
        parameters, P2, GA, cost, reason = evol_dd(parameters,"bre")
        hx_csv[j] = parameters["h_x"]
        hy_csv[j] = parameters["h_y"]
        s_csv[j] = s
        ds_csv[j] = ds    
        P2_csv[j] = P2
        reason_csv[j] = reason
        parameters_csv[j] = parameters
        j += 1
    end
end
csv_data = DataFrame(ds = ds_csv, s = s_csv, h_x = hx_csv, h_y = hy_csv, P2 = P2_csv, parameters = parameters_csv, reason = reason_csv )
CSV.write("result_s_bre_3.csv",csv_data)

Survival with GA

In [ ]:
parameters["b_x"] = 3
parameters["b_y"] = 3
parameters["s_ax"] = 0.5
parameters["s_ay"] = 0.6
parameters["s_u"] = 0.5
parameters["phi"] = 0.1
N = 50
s_csv = zeros(N*3)
ds_csv = zeros(N*3)
hx_csv = zeros(N*3)
hy_csv = zeros(N*3)
P2_csv = zeros(N*3)
parameters_csv = Array{Dict}(undef, N*3)
reason_csv = Array{String}(undef, N*3)
j = 1
for s in [0.3,0.5,0.7]
    parameters["s_x"] = s
    for ds in range(0,0.2,N)
        parameters["s_y"] = parameters["s_x"]+ds
        parameters, P2, GA, cost, reason = evol_dd(parameters,"sub")
        hx_csv[j] = parameters["h_x"]
        hy_csv[j] = parameters["h_y"]
        s_csv[j] = s
        ds_csv[j] = ds    
        P2_csv[j] = P2
        reason_csv[j] = reason
        parameters_csv[j] = parameters
        j += 1
    end
end
csv_data = DataFrame(ds = ds_csv, s = s_csv, h_x = hx_csv, h_y = hy_csv, P2 = P2_csv, parameters = parameters_csv, reason = reason_csv )
CSV.write("result_s_GA_sub_3.csv",csv_data)

In [6]:
parameters["b_x"] = 3
parameters["b_y"] = 3
parameters["s_ax"] = 0.5
parameters["s_ay"] = 0.6
parameters["s_u"] = 0.5
parameters["phi"] = 0.1
N = 50
s_csv = zeros(N*3)
ds_csv = zeros(N*3)
hx_csv = zeros(N*3)
hy_csv = zeros(N*3)
P2_csv = zeros(N*3)
parameters_csv = Array{Dict}(undef, N*3)
reason_csv = Array{String}(undef, N*3)
j = 1
for s in [0.3,0.5,0.7]
    parameters["s_x"] = s
    for ds in range(0,0.2,N)
        parameters["s_y"] = parameters["s_x"]+ds
        parameters, P2, GA, cost, reason = evol_dd(parameters,"bre")
        hx_csv[j] = parameters["h_x"]
        hy_csv[j] = parameters["h_y"]
        s_csv[j] = s
        ds_csv[j] = ds    
        P2_csv[j] = P2
        reason_csv[j] = reason
        parameters_csv[j] = parameters
        j += 1
    end
end
csv_data = DataFrame(ds = ds_csv, s = s_csv, h_x = hx_csv, h_y = hy_csv, P2 = P2_csv, parameters = parameters_csv, reason = reason_csv )
CSV.write("result_s_GA_bre_3.csv",csv_data)

LoadError: SystemError: opening file "result_s_GA_bre_3.csv": Permission denied

Survival for the subordinates

In [7]:
parameters["b_x"] = 3
parameters["b_y"] = 5
parameters["s_x"] = 0.5
parameters["s_y"] = 0.5
parameters["s_u"] = 0.5
parameters["phi"] = 0.1
N = 50
s_csv = zeros(N*3)
ds_csv = zeros(N*3)
hx_csv = zeros(N*3)
hy_csv = zeros(N*3)
P2_csv = zeros(N*3)
parameters_csv = Array{Dict}(undef, N*3)
reason_csv = Array{String}(undef, N*3)
j = 1
for s in [0.5,0.6,0.7]
    parameters["s_u"] = s
    parameters["s_ax"] = s
    for ds in range(0,0.15,N)
        parameters["s_ay"] = parameters["s_ax"]+ds
        parameters, P2, GA, cost, reason = evol_dd(parameters,"sub")
        hx_csv[j] = parameters["h_x"]
        hy_csv[j] = parameters["h_y"]
        s_csv[j] = s
        ds_csv[j] = ds    
        P2_csv[j] = P2
        reason_csv[j] = reason
        parameters_csv[j] = parameters
        j += 1
    end
end
csv_data = DataFrame(ds = ds_csv, s = s_csv, h_x = hx_csv, h_y = hy_csv, P2 = P2_csv, parameters = parameters_csv, reason = reason_csv )
CSV.write("result_say_sub_3.csv",csv_data)

"result_say_sub3.csv"

In [6]:
parameters["b_x"] = 3
parameters["b_y"] = 5
parameters["s_x"] = 0.5
parameters["s_y"] = 0.5
parameters["s_u"] = 0.5
parameters["phi"] = 0.1
N = 50
s_csv = zeros(N*3)
ds_csv = zeros(N*3)
hx_csv = zeros(N*3)
hy_csv = zeros(N*3)
P2_csv = zeros(N*3)
parameters_csv = Array{Dict}(undef, N*3)
reason_csv = Array{String}(undef, N*3)
j = 1
for s in [0.5,0.6,0.7]
    parameters["s_u"] = s
    parameters["s_ax"] = s
    for ds in range(0,0.15,N)
        parameters["s_ay"] = parameters["s_ax"]+ds
        parameters, P2, GA, cost, reason = evol_dd(parameters,"bre")
        hx_csv[j] = parameters["h_x"]
        hy_csv[j] = parameters["h_y"]
        s_csv[j] = s
        ds_csv[j] = ds    
        P2_csv[j] = P2
        reason_csv[j] = reason
        parameters_csv[j] = parameters
        j += 1
    end
end
csv_data = DataFrame(ds = ds_csv, s = s_csv, h_x = hx_csv, h_y = hy_csv, P2 = P2_csv, parameters = parameters_csv, reason = reason_csv )
CSV.write("result_say_bre_3.csv",csv_data)

"result_say_bre3.csv"

## Supplementary data:

In [11]:
parameters["b_x"] = 3
parameters["b_y"] = 3
parameters["s_x"] = 0.5
parameters["s_y"] = 0.5
parameters["s_u"] = 0.5
parameters["phi"] = 0.1
N = 10
s_csv = zeros(N*3)
ds_csv = zeros(N*3)
hx_csv = zeros(N*3)
hy_csv = zeros(N*3)
P2_csv = zeros(N*3)
parameters_csv = Array{Dict}(undef, N*3)
reason_csv = Array{String}(undef, N*3)
j = 1
for s in [0.5,0.6,0.7]
    parameters["s_u"] = s
    for ds in range(0,0.2,N)
        parameters["s_ax"] = parameters["s_u"]+ds
        parameters["s_ay"] = parameters["s_u"]+ds
        parameters, P2, GA, cost, reason = evol_dd(parameters,"sub")
        hx_csv[j] = parameters["h_x"]
        hy_csv[j] = parameters["h_y"]
        s_csv[j] = s
        ds_csv[j] = ds    
        P2_csv[j] = P2
        reason_csv[j] = reason
        parameters_csv[j] = parameters
        j += 1
    end
end
csv_data = DataFrame(ds = ds_csv, s = s_csv, h_x = hx_csv, h_y = hy_csv, P2 = P2_csv, parameters = parameters_csv, reason = reason_csv )
CSV.write("result_sa_sub_3.csv",csv_data)

"result_p_sa_sub.csv"

In [ ]:
parameters["b_x"] = 3
parameters["b_y"] = 3
parameters["s_x"] = 0.5
parameters["s_y"] = 0.5
parameters["s_u"] = 0.5
parameters["phi"] = 0.1
N = 10
s_csv = zeros(N*3)
ds_csv = zeros(N*3)
hx_csv = zeros(N*3)
hy_csv = zeros(N*3)
P2_csv = zeros(N*3)
parameters_csv = Array{Dict}(undef, N*3)
reason_csv = Array{String}(undef, N*3)
j = 1
for s in [0.5,0.6,0.7]
    parameters["s_u"] = s
    for ds in range(0,0.2,N)
        parameters["s_ax"] = parameters["s_u"]+ds
        parameters["s_ay"] = parameters["s_u"]+ds
        parameters, P2, GA, cost, reason = evol_dd(parameters,"sub")
        hx_csv[j] = parameters["h_x"]
        hy_csv[j] = parameters["h_y"]
        s_csv[j] = s
        ds_csv[j] = ds    
        P2_csv[j] = P2
        reason_csv[j] = reason
        parameters_csv[j] = parameters
        j += 1
    end
end
csv_data = DataFrame(ds = ds_csv, s = s_csv, h_x = hx_csv, h_y = hy_csv, P2 = P2_csv, parameters = parameters_csv, reason = reason_csv )
CSV.write("result_sa_bre_3.csv",csv_data)

Breeder-subordinates conflict: phi and fecundity

In [12]:
parameters["s_ax"] = 0.5
parameters["s_ay"] = 0.6
parameters["s_u"] = 0.5
parameters["s_x"] = 0.5
parameters["s_y"] = 0.5
parameters["b_x"] = 3
parameters["b_y"] = 3
parameters["phi"] = 0.1
N = 30
phi_csv = zeros(N*3)
db_csv = zeros(N*3)
hx_csv = zeros(N*3)
hy_csv = zeros(N*3)
P_csv = zeros(N*3)
parameters_csv = Array{Dict}(undef, N*3)
reason_csv = Array{String}(undef, N*3)
GA_csv = zeros(N*3)
cost_csv = zeros(N*3)
j = 1
for db in [2,3,4]
    parameters["b_y"] = parameters["b_x"]+db
    for phi in range(0,1,N)
    parameters["phi"] = phi
        parameters, P2, GA, cost, reason = evol_dd(parameters,"sub")
        hx_csv[j] = parameters["h_x"]
        hy_csv[j] = parameters["h_y"]
        phi_csv[j] = phi
        db_csv[j] = db
        GA_csv[j] = GA
        cost_csv[j] = cost
        reason_csv[j] = reason
        parameters_csv[j] = parameters
        j += 1
    end
end
csv_data = DataFrame(db = db_csv, phi = phi_csv, h_x = hx_csv, h_y = hy_csv, cost=cost_csv, GA = GA_csv, parameters = parameters_csv, reason = reason_csv )
CSV.write("result_phi_b_GA_sub_3.csv",csv_data)

LoadError: SystemError: opening file "result_phi_b_GA_bre3.csv": Permission denied

In [ ]:
parameters["s_ax"] = 0.5
parameters["s_ay"] = 0.6
parameters["s_u"] = 0.5
parameters["s_x"] = 0.5
parameters["s_y"] = 0.5
parameters["b_x"] = 3
parameters["b_y"] = 3
parameters["phi"] = 0.1
N = 30
phi_csv = zeros(N*3)
db_csv = zeros(N*3)
hx_csv = zeros(N*3)
hy_csv = zeros(N*3)
P_csv = zeros(N*3)
parameters_csv = Array{Dict}(undef, N*3)
reason_csv = Array{String}(undef, N*3)
GA_csv = zeros(N*3)
cost_csv = zeros(N*3)
j = 1
for db in [2,3,4]
    parameters["b_y"] = parameters["b_x"]+db
    for phi in range(0,1,N)
    parameters["phi"] = phi
        parameters, P2, GA, cost, reason = evol_dd(parameters,"bre")
        hx_csv[j] = parameters["h_x"]
        hy_csv[j] = parameters["h_y"]
        phi_csv[j] = phi
        db_csv[j] = db
        GA_csv[j] = GA
        cost_csv[j] = cost
        reason_csv[j] = reason
        parameters_csv[j] = parameters
        j += 1
    end
end
csv_data = DataFrame(db = db_csv, phi = phi_csv, h_x = hx_csv, h_y = hy_csv, cost=cost_csv, GA = GA_csv, parameters = parameters_csv, reason = reason_csv )
CSV.write("result_phi_b_GA_bre_3.csv",csv_data)

Breeder-subordinate conflict: survival and phi

In [10]:
parameters["s_ax"] = 0.5
parameters["s_ay"] = 0.6
parameters["s_u"] = 0.5
parameters["s_x"] = 0.5
parameters["s_y"] = 0.5
parameters["b_x"] = 3
parameters["b_y"] = 3
parameters["phi"] = 0.1
N = 10
phi_csv = zeros(N*3)
ds_csv = zeros(N*3)
hx_csv = zeros(N*3)
hy_csv = zeros(N*3)
P_csv = zeros(N*3)
parameters_csv = Array{Dict}(undef, N*3)
reason_csv = Array{String}(undef, N*3)
GA_csv = zeros(N*3)
cost_csv = zeros(N*3)
j = 1
for ds in [0.2,0.3,0.4]
    parameters["s_y"] = parameters["s_x"]+ds
    for phi in range(0,1,N)
    parameters["phi"] = phi
        parameters, P2, GA, cost, reason = evol_dd(parameters,"sub")
        hx_csv[j] = parameters["h_x"]
        hy_csv[j] = parameters["h_y"]
        phi_csv[j] = phi
        ds_csv[j] = ds
        GA_csv[j] = GA
        cost_csv[j] = cost
        reason_csv[j] = reason
        parameters_csv[j] = parameters
        j += 1
    end
end
csv_data = DataFrame(ds = ds_csv, phi = phi_csv, h_x = hx_csv, h_y = hy_csv, cost=cost_csv, GA = GA_csv, parameters = parameters_csv, reason = reason_csv )
CSV.write("result_phi_s_GA_sub_3.csv",csv_data)

"result_phi_s_GA_sub_4.csv"

In [10]:
parameters["s_ax"] = 0.5
parameters["s_ay"] = 0.6
parameters["s_u"] = 0.5
parameters["s_x"] = 0.5
parameters["s_y"] = 0.5
parameters["b_x"] = 3
parameters["b_y"] = 3
parameters["phi"] = 0.1
N = 20
phi_csv = zeros(N*3)
ds_csv = zeros(N*3)
hx_csv = zeros(N*3)
hy_csv = zeros(N*3)
P_csv = zeros(N*3)
parameters_csv = Array{Dict}(undef, N*3)
reason_csv = Array{String}(undef, N*3)
GA_csv = zeros(N*3)
cost_csv = zeros(N*3)
j = 1
for ds in [0.2,0.3,0.4]
    parameters["s_y"] = parameters["s_x"]+ds
    for phi in range(0,1,N)
    parameters["phi"] = phi
        parameters, P2, GA, cost, reason = evol_dd(parameters,"bre")
        hx_csv[j] = parameters["h_x"]
        hy_csv[j] = parameters["h_y"]
        phi_csv[j] = phi
        ds_csv[j] = ds
        GA_csv[j] = GA
        cost_csv[j] = cost
        reason_csv[j] = reason
        parameters_csv[j] = parameters
        j += 1
    end
end
csv_data = DataFrame(ds = ds_csv, phi = phi_csv, h_x = hx_csv, h_y = hy_csv, cost=cost_csv, GA = GA_csv, parameters = parameters_csv, reason = reason_csv )
CSV.write("result_phi_s_GA_bre3.csv",csv_data)

"result_phi_s_GA_bre3.csv"